In [39]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import platform
platform.system()

if platform.system() == 'Darwin': # Mac 환경 폰트 설정
    plt.rc('font', family='AppleGothic')
elif platform.system() == 'Windows': # Windows 환경 폰트 설정
    plt.rc('font', family='Malgun Gothic')

plt.rc('axes', unicode_minus=False) # 마이너스 폰트 설정
plt.rc('axes', labelsize=30)   # x,y축 label 폰트 크기


# 글씨 선명하게 출력하는 설정
%config InlineBackend.figure_format = 'retina'

In [48]:
import os
import random
import time

from itertools import combinations

from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, mean_absolute_percentage_error
from sklearn.model_selection import train_test_split

import tensorflow as tf
import tensorflow_addons as tfa

import seaborn as sns

In [49]:
dataframe = pd.read_csv("C:/Users/wjddu/git/Domestic-Solar-Power-Output-Prediction/처리 data.csv")

In [50]:
dataframe

,date,year,month,day,hour,발전량,기온,습도,이슬점 온도,일사량,구름양
0,2016-12-13 14:00,2016,12,13,14,140.631,4.3,77,0.6,100.000,0
1,2016-12-13 15:00,2016,12,13,15,85.793,4.3,80,1.1,72.222,0
2,2016-12-13 16:00,2016,12,13,16,32.590,3.8,80,0.6,55.556,0
3,2016-12-13 17:00,2016,12,13,17,0.154,3.4,80,0.2,13.889,0
4,2016-12-13 18:00,2016,12,13,18,0.000,2.6,86,0.4,2.778,0
...,...,...,...,...,...,...,...,...,...,...,...
22487,2020-12-31 19:00,2020,12,31,19,0.000,-0.9,65,-6.6,0.000,8
22488,2020-12-31 20:00,2020,12,31,20,0.000,-1.5,66,-7.0,0.000,8
22489,2020-12-31 21:00,2020,12,31,21,0.000,-1.9,75,-5.7,0.000,8
22490,2020-12-31 22:00,2020,12,31,22,0.000,-2.7,73,-6.8,0.000,8


In [51]:
dataframe.describe()

,year,month,day,hour,발전량,기온,습도,이슬점 온도,일사량,구름양
count,22492.000000,22492.000000,22492.000000,22492.000000,22492.000000,22492.000000,22492.000000,22492.000000,22492.000000,22492.000000
mean,2018.496043,6.488129,15.872088,11.310421,184.657806,10.960359,68.475991,4.509421,114.112308,2.873466
std,1.164338,3.750345,8.651306,7.159485,299.821819,7.860266,21.518554,7.847142,195.508104,3.880111
min,2016.000000,1.000000,1.000000,0.000000,0.000000,-8.400000,0.000000,-10.400000,0.000000,0.000000
25%,2017.000000,3.000000,8.000000,5.000000,0.000000,4.700000,52.000000,-1.800000,0.000000,0.000000
50%,2019.000000,5.000000,16.000000,11.000000,0.000000,10.800000,70.000000,4.600000,0.000000,0.000000
75%,2020.000000,10.000000,23.000000,18.000000,276.796750,16.900000,88.000000,11.400000,156.250250,7.000000
max,2020.000000,12.000000,31.000000,23.000000,1029.248000,39.200000,100.000000,17.400000,955.556000,10.000000


In [52]:
X_features = ['month', 'hour', '기온', '습도', '이슬점 온도', '일사량', '구름양']
y_feature = ['발전량']

X = dataframe[X_features]
y = dataframe[y_feature]

X.shape, y.shape

((22492, 7), (22492, 1))

In [53]:
x_scaler = MinMaxScaler()
y_scaler = MinMaxScaler()

X_norm, y_norm = X.copy(), y.copy()
X_norm.iloc[:, 2:] = x_scaler.fit_transform(X_norm.iloc[:, 2:])
y_norm.iloc[:, :] = y_scaler.fit_transform(y_norm.iloc[:, :])

In [54]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

dataframe[['발전량']] = scaler.fit_transform(dataframe[['발전량']])
dataframe[['기온']] = scaler.fit_transform(dataframe[['기온']])
dataframe[['습도']] = scaler.fit_transform(dataframe[['습도']])
dataframe[['이슬점 온도']] = scaler.fit_transform(dataframe[['이슬점 온도']])
dataframe[['일사량']] = scaler.fit_transform(dataframe[['일사량']])
dataframe[['구름양']] = scaler.fit_transform(dataframe[['구름양']])

In [55]:
dataframe.describe()

,year,month,day,hour,발전량,기온,습도,이슬점 온도,일사량,구름양
count,22492.000000,22492.000000,22492.000000,22492.000000,22492.000000,22492.000000,22492.000000,22492.000000,22492.000000,22492.000000
mean,2018.496043,6.488129,15.872088,11.310421,0.179410,0.406730,0.684760,0.536310,0.119420,0.287347
std,1.164338,3.750345,8.651306,7.159485,0.291302,0.165132,0.215186,0.282271,0.204601,0.388011
min,2016.000000,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2017.000000,3.000000,8.000000,5.000000,0.000000,0.275210,0.520000,0.309353,0.000000,0.000000
50%,2019.000000,5.000000,16.000000,11.000000,0.000000,0.403361,0.700000,0.539568,0.000000,0.000000
75%,2020.000000,10.000000,23.000000,18.000000,0.268931,0.531513,0.880000,0.784173,0.163518,0.700000
max,2020.000000,12.000000,31.000000,23.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [56]:
dataframe.to_csv("최종 data.csv", header=True, index = False, encoding='cp949')